### Hello World
Basic text Hello World example in TensorFlow.


In [124]:
import tensorflow as tf
import numpy as np
hello = tf.constant('Hello, TensorFlow!')

sess = tf.Session()
print(sess.run(hello))
sess.close()

b'Hello, TensorFlow!'


### Hello World with a Matrix
Hello World with a Matrix from NumPy used to initialize a TensorFlow Variable.
Before using the variable, we need to run the initializer operation.

In [125]:
y_np = np.array([[1,2,3], [4,5,6]])
y_tf = tf.Variable(y_np)
# matrix1 = tf.constant([[3., 3.]])  # matrix constant

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
print(sess.run(y_tf))
sess.close()

[[1 2 3]
 [4 5 6]]


## Sessions Run and Eval
* When using "with", there is a default session and closing is automatic
* eval() uses the default session (which is established inside the "with")
* A "sesssion" parameter must be supplied to eval() when not within a "with".

In [126]:
const = tf.fill([2, 3], 42, "test_tensor")
with tf.Session() as sess:
    val = sess.run(const)
    print(val)
    
with tf.Session() as sess:
    val2 = const.eval() # uses the default session
    print(val2)
    
sess = tf.Session()
val3 = const.eval(session=sess)
print(val3)
sess.close()

[[42 42 42]
 [42 42 42]]
[[42 42 42]
 [42 42 42]]
[[42 42 42]
 [42 42 42]]


### Saving and Restoring Variables
Shows how to save and restore a counter that increments with each execution.

In [128]:
import os.path
path = "/tmp/model_counter_c.ckpt"
counter = tf.Variable(1.0)
increment = tf.assign(counter, counter + 1)
saver = tf.train.Saver({"my_counter": counter})  # List of parameters to save is optional
with tf.Session() as sess:
    # Restore previously saved variables
    try:
        saver.restore(sess, path)
        print("Restored counter=", sess.run(counter))
    except Exception as e:
        sess.run(tf.global_variables_initializer())
        print("Initialized counter")
    
    # Increment the counter
    increment.eval()  # This is just alternate syntax for: sess.run(increment)
    value = sess.run(counter)
    
    # Save updated value of the variables
    saver.save(sess, path)
    print("Saved counter=", value)
 

INFO:tensorflow:Restoring parameters from /tmp/model_counter_c.ckpt
Restored counter= 16.0
Saved counter= 17.0


### Cross Entropy with Logits
Given the ideal probability distribution of labels, find the crossentropy from generating y_logits <br/>
Note: Logits are in range of 0 to 1

In [129]:
with tf.Session() as sess:
    y_logits_np = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]) # the estimated values scaled between 0 to 1
    y_logits = tf.Variable(y_logits_np, "y_logits")
    
    labels_np = np.array([[.2, .3, .5], [0.9, 0.1, 0.0]])  # the ideal probabilities
    labels = tf.Variable(labels_np, "labels")
    
    # Method 1: Using TF built-in function
    y_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_logits, labels=labels)
    
    # Method 2: Showing underlying math
    y_softmax = tf.nn.softmax(y_logits) # does not reduce dimensionality
    y_entropy2 = -tf.reduce_sum(labels * tf.log(y_softmax), 1) # take the sum of the products along axis=1
        
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    y_entropy_val, y_entropy_val2 = sess.run([y_entropy, y_entropy2])

    for entropy, entropy2, rowIndex in zip(y_entropy_val, y_entropy_val2, range(y_entropy_val.size)):
        print("Entropy[%d] %.2f == %.2f" % (rowIndex, entropy, entropy2))
    

Entropy[0] 1.11 == 1.11
Entropy[1] 2.31 == 2.31


### Entropy for sparse distribution
When the ideal distribution has a single correct answer, then use sparse function for more efficiency.


In [130]:
with tf.Session() as sess:
    y_logits_np = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]) # the estimated values scaled between 0 to 1
    y_logits = tf.Variable(y_logits_np, "y_logits")
    
    label_indexes_np = np.array([2, 1])  # the ideal probability indexes
    label_indexes = tf.Variable(label_indexes_np, "label_indexes")

    # labels_np = np.array([[0.0, 0, 1], [0.0, 1, 0]])  # the ideal probabilities (notice they are one-hot)
    labels_np = np.zeros((2, 3))
    labels_np[np.arange(2), label_indexes_np] = 1;
    labels = tf.Variable(labels_np, "labels")


    # Method 1: Using TF built-in SPARSE function
    y_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_logits, labels=label_indexes)
    # Method 2: Using TF built-in non-sparse function (requires labels to be one-hot)
    y_entropy2 = tf.nn.softmax_cross_entropy_with_logits(logits=y_logits, labels=labels)
        
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    y_entropy_val, y_entropy_val2 = sess.run([y_entropy, y_entropy2])

    for entropy, entropy2, rowIndex in zip(y_entropy_val, y_entropy_val2, range(y_entropy_val.size)):
        print("Entropy[%d] %.2f == %.2f" % (rowIndex, entropy, entropy2))

Entropy[0] 0.41 == 0.41
Entropy[1] 1.41 == 1.41
